# Projeto

## Data Preparation

- Eliminar Colunas IDentificadoras e tornar ID como PK

- Eliminar Erros de Input
    - Format 'Age' & Check SSN pattern
    - Negative Numeric Variables: 'Age', 'Num_Bank_Accounts', 'NumofLoan'
    - Age Restriction (GREATER than 0 && LESS than 150)
<br><br>
- Eliminar 9 Records de 'Monthly_Balance' igual a -3.33333E+26

- Trabalhar com 'Credit_History_Age' como variavel numerica 

- Variable Enconding of 'Age' and 'Credit_History_Age'

In [1]:
import pandas as pd
import re
import numpy as np
import dslabs_functions as dslab

file_tag = 'credit_score'
filename = 'class_credit_score.csv'
original_dataset: pd.DataFrame = pd.read_csv(filename, index_col='ID', sep=',', decimal='.', na_values='')

# Drop IDentifying Columns
dataset_cleaned = original_dataset.drop(columns=['Customer_ID', 'Name', 'SSN'], errors='ignore')

# Erase Input Errors
## Check and Clean 'SSN' & 'Age' Column
pattern = r'^\d{3}-\d{2}-\d{4}$'  # Replace this with your SSN regex pattern
#dataset_cleaned['SSN'] = dataset_cleaned['SSN'].apply(lambda x: x if re.match(pattern, str(x)) else np.nan)
dataset_cleaned['Age'] = dataset_cleaned['Age'].astype(str).str.replace('_', '', regex=False)

## Transfrom 'Age' from SIMBOLIC to NUMERIC
dataset_cleaned['Age'] = pd.to_numeric(dataset_cleaned['Age'], errors='coerce')

## Invalid Numeric Variables 'Age', 'Num_Bank_Accounts', 'NumofLoan' Column
dataset_cleaned['Age'] = dataset_cleaned['Age'].apply(lambda x: np.nan if x == -500 or x >= 150 else x)
dataset_cleaned['Num_Bank_Accounts'] = dataset_cleaned['Num_Bank_Accounts'].apply(lambda x: x if x >= 0 else np.nan)
dataset_cleaned['NumofLoan'] = dataset_cleaned['NumofLoan'].apply(lambda x: x if x >= 0 else np.nan)

# Monthly_Balance 
dataset_cleaned['MonthlyBalance'] = dataset_cleaned['MonthlyBalance'].apply(lambda x: x if x >= 0 else np.nan)

## Transform 'Credit_History_Age' from SIMBOLIC to NUMERIC
hist_age = dataset_cleaned["Credit_History_Age"]
hist_vals = []

for val in hist_age:
    matches = re.findall(r'(\d+) Years and (\d+) Months', str(val))
    if not matches:  # Use `not matches` instead of `matches == []`
        hist_vals.append(np.nan)
    else:
        hist_vals.append(float(matches[0][0]) + (float(matches[0][1]) / 12))

dataset_cleaned["Credit_History_Age"] = hist_vals

### Drop 'Credit_History_Age' as OBJECT 
object_columns = dataset_cleaned.select_dtypes(include=['object']).columns.tolist()
if "CreditHistoryAge" in object_columns:
    dataset_cleaned.drop(columns=["CreditHistoryAge"], inplace=True)

# Get Variables Type Right (Object to Category)
variables_types: dict[str, list] = dslab.get_variable_types(dataset_cleaned)
numeric: list[str] = variables_types["numeric"]
symbolic: list[str] = variables_types["symbolic"]
binary: list[str] = variables_types["binary"]

print(variables_types)

dataset_cleaned[symbolic] = dataset_cleaned[symbolic].apply(lambda x: x.astype("category"))
print(dataset_cleaned.dtypes)

{'numeric': ['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'NumofLoan', 'Delay_from_due_date', 'NumofDelayedPayment', 'ChangedCreditLimit', 'NumCreditInquiries', 'OutstandingDebt', 'CreditUtilizationRatio', 'Credit_History_Age', 'TotalEMIpermonth', 'Amountinvestedmonthly', 'MonthlyBalance'], 'binary': ['Credit_Score'], 'date': [], 'symbolic': ['Month', 'Occupation', 'Type_of_Loan', 'CreditMix', 'Payment_of_Min_Amount', 'Payment_Behaviour']}
Month                     category
Age                        float64
Occupation                category
Annual_Income              float64
Monthly_Inhand_Salary      float64
Num_Bank_Accounts          float64
Num_Credit_Card              int64
Interest_Rate                int64
NumofLoan                  float64
Type_of_Loan              category
Delay_from_due_date          int64
NumofDelayedPayment        float64
ChangedCreditLimit         float64
NumCreditInquiries         float64
Credi

### Variable Enconding

- *'Age' & 'Credit_History_Age' feitos na 1ª célula*

1. 'Credit_Score
    - Ordinal Enconding (easiest one)
2. 'Month'<br>
3. CreditMix'
4. 'Payment_of_Min_Amount
5. 'Payment_Behaviour'

6. 'Occupation'
7. 'Type_of_Loan'

In [15]:
# Enconding 'Data_Score' Binary - Ordinal Enconding (easiest one)
variables_types

yes_no: dict[str, int] = {"no": 0, "No": 0, "yes": 1, "Yes": 1}
credit_score_values: dict[str, int] = {"Poor": 0, "Good": 1}

encoding: dict[str, dict[str, int]] = {
    "Credit_Score": credit_score_values,
}

data_encoded: pd.DataFrame = dataset_cleaned.replace(encoding, inplace=False)
data_encoded

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,NumofLoan,Type_of_Loan,...,CreditMix,OutstandingDebt,CreditUtilizationRatio,Credit_History_Age,Payment_of_Min_Amount,TotalEMIpermonth,Amountinvestedmonthly,Payment_Behaviour,MonthlyBalance,Credit_Score
ID,,,,,,,,,,,,,,,,,,,,,
0x1602,January,23.0,Scientist,19114.12,1824.843333,3.0,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,NaN,809.98,26.822620,22.083333,No,49.574949,80.415295,High_spent_Small_value_payments,312.494089,1
0x1603,February,23.0,Scientist,19114.12,NaN,3.0,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,Good,809.98,31.944960,NaN,No,49.574949,118.280222,Low_spent_Large_value_payments,284.629163,1
0x1604,March,NaN,Scientist,19114.12,NaN,3.0,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,Good,809.98,28.609352,22.250000,No,49.574949,81.699521,Low_spent_Medium_value_payments,331.209863,1
0x1605,April,23.0,Scientist,19114.12,NaN,3.0,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,Good,809.98,31.377862,22.333333,No,49.574949,199.458074,Low_spent_Small_value_payments,223.451310,1
0x1606,May,23.0,Scientist,19114.12,1824.843333,3.0,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,Good,809.98,24.797347,22.416667,No,49.574949,41.420153,High_spent_Medium_value_payments,341.489231,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0x25fe9,April,25.0,Mechanic,39628.99,3359.415833,4.0,6,7,2.0,"Auto Loan, and Student Loan",...,NaN,502.38,34.663572,31.500000,No,35.104023,60.971333,High_spent_Large_value_payments,479.866228,0
0x25fea,May,25.0,Mechanic,39628.99,3359.415833,4.0,6,7,2.0,"Auto Loan, and Student Loan",...,NaN,502.38,40.565631,31.583333,No,35.104023,54.185950,High_spent_Medium_value_payments,496.651610,0
0x25feb,June,25.0,Mechanic,39628.99,3359.415833,4.0,6,5729,2.0,"Auto Loan, and Student Loan",...,Good,502.38,41.255522,31.666667,No,35.104023,24.028477,High_spent_Large_value_payments,516.809083,0


In [14]:
# Collect Individual values for each symbolic var, in order to encode the rest
data_encoded

for v in variables_types["symbolic"]:
    print(v, data_encoded[v].unique())

Month ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August']
Categories (8, object): ['April', 'August', 'February', 'January', 'July', 'June', 'March', 'May']
Occupation ['Scientist', NaN, 'Teacher', 'Engineer', 'Entrepreneur', ..., 'Accountant', 'Musician', 'Mechanic', 'Writer', 'Architect']
Length: 16
Categories (15, object): ['Accountant', 'Architect', 'Developer', 'Doctor', ..., 'Musician', 'Scientist', 'Teacher', 'Writer']
Type_of_Loan ['Auto Loan, Credit-Builder Loan, Personal Loan..., 'Credit-Builder Loan', 'Auto Loan, Auto Loan, and Not Specified', 'Not Specified', NaN, ..., 'Auto Loan, Payday Loan, Auto Loan, Student Lo..., 'Home Equity Loan, Payday Loan, Not Specified,..., 'Home Equity Loan, Auto Loan, Auto Loan, and A..., 'Payday Loan, Student Loan, Mortgage Loan, and..., 'Personal Loan, Auto Loan, Mortgage Loan, Stud...]
Length: 6261
Categories (6260, object): ['Auto Loan', 'Auto Loan, Auto Loan, Auto Loan, Auto Loan, C..., 'Auto Loan, Auto Loan, Auto L